In [2]:
import torch
from PIL import Image
from torchvision import transforms, models
from collections import OrderedDict, Counter
from functools import partial
from dataclasses import dataclass
from typing import Tuple, List
from copy import deepcopy
from math import prod
from sys import version
version

'3.9.5 (default, Aug 29 2021, 19:01:31) \n[GCC 9.3.0]'

In [3]:
model_dict = {
    'ssd': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd'),
    'lenet': torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True),
    'yolov5s': torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True),
    'yolov5m': torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True),
    'yolov5l': torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True),
    'yolov5x': torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True),
    'alexnet': models.alexnet(pretrained=True, progress=True),
    'vgg_11': models.vgg11(pretrained=True, progress=True),
    'vgg_13': models.vgg13(pretrained=True, progress=True),
    'vgg_16': models.vgg16(pretrained=True, progress=True),
    'vgg_19': models.vgg19(pretrained=True, progress=True),
    'vgg_11_bn': models.vgg11_bn(pretrained=True, progress=True),
    'vgg_13_bn': models.vgg13_bn(pretrained=True, progress=True),
    'vgg_16_bn': models.vgg16_bn(pretrained=True, progress=True),
    'vgg_19_bn': models.vgg19_bn(pretrained=True, progress=True),
    'resnet_18': models.resnet18(pretrained=True, progress=True),
    'resnet_34': models.resnet34(pretrained=True, progress=True),
    'resnet_50': models.resnet50(pretrained=True, progress=True),
    'resnet_101': models.resnet101(pretrained=True, progress=True),
    'resnet_152': models.resnet152(pretrained=True, progress=True),
    'squeezenet_1_0': models.squeezenet1_1(pretrained=True, progress=True),
    'squeezenet_1_1': models.squeezenet1_0(pretrained=True, progress=True),
    'densenet_121': models.densenet121(pretrained=True, progress=True),
    'densenet_169': models.densenet169(pretrained=True, progress=True),
    'densenet_201': models.densenet201(pretrained=True, progress=True),
    'densenet_161': models.densenet161(pretrained=True, progress=True),
    'inception_v3': models.inception_v3(pretrained=True, progress=True),
    'googlenet': models.googlenet(pretrained=True, progress=True),
    'shufflenet_v2_x0_5': models.shufflenet_v2_x0_5(pretrained=True, progress=True),
    'shufflenet_v2_x1_0': models.shufflenet_v2_x1_0(pretrained=True, progress=True),
    'mobilenet_v2': models.mobilenet_v2(pretrained=True, progress=True),
    'mobilenet_v3_large': models.mobilenet_v3_large(pretrained=True, progress=True),
    'mobilenet_v3_small': models.mobilenet_v3_small(pretrained=True, progress=True),
    'resnext_50_32x4d': models.resnext50_32x4d(pretrained=True, progress=True),
    'resnext_101_32x8d': models.resnext101_32x8d(pretrained=True, progress=True),
    'wide_resnet_50_2': models.wide_resnet50_2(pretrained=True, progress=True),
    'wide_resnet_101_2': models.wide_resnet101_2(pretrained=True, progress=True),
    'mnasnet0_5': models.mnasnet0_5(pretrained=True, progress=True),
    'mnasnet1_0': models.mnasnet1_0(pretrained=True, progress=True),
}


Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/sultan/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 308 layers, 21356877 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 392 layers, 47025981 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics

In [4]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [5]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg'
]
inputs = [utils.prepare_input(uri) for uri in uris]
ssd_input_batch = utils.prepare_tensor(inputs)

In [6]:
def get_next_conv_layers(model):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.modules.conv.Conv2d):
            yield (name, module)

In [7]:
@dataclass
class LayerDimensions:
    kernel_size: Tuple[int, int]
    stride: Tuple[int, int]
    padding: Tuple[int, int]
    input_size: List[int]
    output_size: List[int]


class ModelStatCollector:
    def __init__(self):
        self.model_stats = OrderedDict()
        self.hooks = []

    def __extract_stats(self, name, module, input, output):
        self.model_stats[name] = LayerDimensions(module.kernel_size, module.stride, module.padding, input_size=list(
            input[0].size()), output_size=list(output[0].size()))

    def __attach_collection_hooks_to_model(self, model):

        for name, conv_layer in get_next_conv_layers(model):
            layer_collector = partial(self.__extract_stats, name)
            self.hooks.append(
                conv_layer.register_forward_hook(layer_collector))

    def __detach_stats_collection_hooks(self):
        for hook in self.hooks:
            hook.remove()

    def __reset(self):
        self.model_stats = {}
        self.hooks = []

    def collect_stats_from_model(self, model, input_batch):
        self.__attach_collection_hooks_to_model(model)
        model.eval()
        # move the input and model to GPU for speed if available
        with torch.no_grad():
            model(input_batch)
        self.__detach_stats_collection_hooks()
        collected_stats = deepcopy(self.model_stats)
        self.__reset()
        return collected_stats


class ModelStatAnalyser:
    @ classmethod
    def get_kernel_stats(cls, model_stats):
        kernel_size_counter = Counter()
        stride_counter_dict = {}
        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            stride = layer.stride
            kernel_size_counter.update(str(kernel_size[0]))
            if str(kernel_size[0]) not in stride_counter_dict:
                stride_counter_dict[str(kernel_size[0])] = Counter()
            stride_counter_dict[str(kernel_size[0])].update(str(stride[0]))
        return kernel_size_counter

    @ classmethod
    def get_intermediate_layer_sizes(cls, model_stats):
        intermediate_layer_sizes = [
            prod(layer.input_size) for layer in model_stats.values()]
        intermediate_layer_sizes.append(
            prod(list(model_stats.values())[-1].output_size))
        return intermediate_layer_sizes

    @ classmethod
    def get_intermediate_layer_size_bounds(cls, model_stats):
        return (max(cls.get_intermediate_layer_sizes(model_stats)),
                min(cls.get_intermediate_layer_sizes(model_stats)))

    @ classmethod
    def get_ub_input_size(cls, model_stats):
        return max([prod(layer.kernel_size[0]) for layer in model_stats.values()])

    @ classmethod
    def get_stride_stats(cls, model_stats):
        stride_counter_dict = {}

        for layer in model_stats.values():
            kernel_size = layer.kernel_size
            stride = layer.stride
            if str(kernel_size[0]) not in stride_counter_dict:
                stride_counter_dict[str(kernel_size[0])] = Counter()
            stride_counter_dict[str(kernel_size[0])].update(str(stride[0]))
        return stride_counter_dict


In [22]:
collector = ModelStatCollector()
stats_dict = {}
raw_stats_dict = {}
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    ssd_input_batch = ssd_input_batch.to('cuda')
for model_name, model in model_dict.items():
    print('Analysing {}'.format(model_name))
    model.to('cuda')
    if model_name == 'ssd':
        model_stats = collector.collect_stats_from_model(model, ssd_input_batch)
    else:
        model_stats = collector.collect_stats_from_model(model, input_batch)
        
    model.to('cpu')
    raw_stats_dict[model_name] = model_stats
    stats_dict[model_name] = {'kernel': ModelStatAnalyser.get_kernel_stats(model_stats),
                              'stride': ModelStatAnalyser.get_stride_stats(model_stats),
                              'intermediate_layer_bounds': ModelStatAnalyser.get_intermediate_layer_size_bounds(model_stats)}

Analysing ssd
Analysing lenet
Analysing yolov5s
Analysing yolov5m
Analysing yolov5l
Analysing yolov5x
Analysing alexnet
Analysing vgg_11
Analysing vgg_13
Analysing vgg_16
Analysing vgg_19
Analysing vgg_11_bn
Analysing vgg_13_bn
Analysing vgg_16_bn
Analysing vgg_19_bn
Analysing resnet_18
Analysing resnet_34
Analysing resnet_50
Analysing resnet_101
Analysing resnet_152
Analysing squeezenet_1_0
Analysing squeezenet_1_1
Analysing densenet_121
Analysing densenet_169
Analysing densenet_201
Analysing densenet_161
Analysing inception_v3
Analysing googlenet
Analysing shufflenet_v2_x0_5
Analysing shufflenet_v2_x1_0
Analysing mobilenet_v2
Analysing mobilenet_v3_large
Analysing mobilenet_v3_small
Analysing resnext_50_32x4d
Analysing resnext_101_32x8d
Analysing wide_resnet_50_2
Analysing wide_resnet_101_2
Analysing mnasnet0_5
Analysing mnasnet1_0


In [50]:
aggregate_kernel_stats = Counter()
for model, stats in stats_dict.items():
    aggregate_kernel_stats += stats['kernel']
ksize, counts = zip(*[(ksize, count) for ksize, count in aggregate_kernel_stats.items()])
total_kernels = sum(counts)
aggregate_kernel_stats_percentages = {ksize: counts/total_kernels for ksize, counts in zip(ksize,counts)}
aggregate_kernel_stats_percentages

{'7': 0.011862396204033215,
 '1': 0.5776986951364176,
 '3': 0.39541320680110714,
 '5': 0.015025701858442072}

In [49]:
aggregate_stride_stats = {}
for model, stats in stats_dict.items():
    for kernel, counter in stats['stride'].items():
        if kernel not in aggregate_stride_stats:
            aggregate_stride_stats[kernel] = Counter()
        aggregate_stride_stats[kernel] += counter

aggregate_stride_stats_percentages = {}
for ksize, stride_counter in aggregate_stride_stats.items():
    total_kernels = sum(stride_counter.values())
    aggregate_stride_stats_percentages[ksize] = {stride: count/total_kernels for stride, count in dict(stride_counter).items()}
aggregate_stride_stats_percentages

{'7': {'2': 0.5666666666666667, '1': 0.43333333333333335},
 '1': {'1': 0.9808087731322824, '2': 0.019191226867717615},
 '3': {'1': 0.91, '2': 0.09},
 '11': {'4': 1.0},
 '5': {'1': 0.7368421052631579, '2': 0.2631578947368421}}

In [ ]:
for name, layers in raw_stats_dict.items():
    for layer_name, layer in layers.items():
        if layer.kernel_size[0] == 1 and layer.stride[0] == 2:
            print("model {}, has kernel of size 1 with stide 2: {}".format(name, layer))

In [59]:
raw_stats_dict['yolov5l']

{'model.model.0.conv.conv': LayerDimensions(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), input_size=[1, 12, 112, 112], output_size=[64, 112, 112]),
 'model.model.1.conv': LayerDimensions(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), input_size=[1, 64, 112, 112], output_size=[128, 56, 56]),
 'model.model.2.cv1.conv': LayerDimensions(kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), input_size=[1, 128, 56, 56], output_size=[64, 56, 56]),
 'model.model.2.m.0.cv1.conv': LayerDimensions(kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), input_size=[1, 64, 56, 56], output_size=[64, 56, 56]),
 'model.model.2.m.0.cv2.conv': LayerDimensions(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), input_size=[1, 64, 56, 56], output_size=[64, 56, 56]),
 'model.model.2.m.1.cv1.conv': LayerDimensions(kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), input_size=[1, 64, 56, 56], output_size=[64, 56, 56]),
 'model.model.2.m.1.cv2.conv': LayerDimensions(kernel_size=(3, 3), stride=(1, 1), padding